In [ ]:
!pip install xgboost tensorflow pandas numpy scikit-learn matplotlib


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load dataset
df = pd.read_csv('/content/complete_solar_data2.csv')

# Convert DATE_TIME to datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], errors='coerce')

# Extract time-based features

df['MONTH'] = df['DATE_TIME'].dt.month
df['DAY'] = df['DATE_TIME'].dt.day
df['HOUR'] = df['DATE_TIME'].dt.hour

# Drop DATE_TIME column
df.drop(columns=['DATE_TIME'], inplace=True)

# Drop NaN values (if any)
df.dropna(inplace=True)

# Define target column
target_column = 'AC_POWER'

# Separate features and target
features = df.drop(columns=[target_column])
target = df[target_column]

# Feature Scaling
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Reshape data for LSTM (Sliding Window Technique)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build Improved LSTM Model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(1, X_train.shape[2]))),
    BatchNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(32, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu')  # Output layer
])

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train Model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr])

# Predictions
y_pred = model.predict(X_test)

# Performance Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate Accuracy using MAPE
mape = np.mean(np.abs((y_test - y_pred.flatten()) / y_test)) * 100
accuracy = 100 - mape  # Accuracy formula

# Print Results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")
print(f"Model Accuracy (100 - MAPE): {accuracy:.2f}%")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


847/847 ━━━━━━━━━━━━━━━━━━━━ 23s 14ms/step - loss: 91516.6953 - val_loss: 17282.4883 - learning_rate: 0.0010
Epoch 2/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 23458.8496 - val_loss: 16965.6426 - learning_rate: 0.0010
Epoch 3/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - loss: 22358.5059 - val_loss: 14773.6660 - learning_rate: 0.0010
Epoch 4/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 21332.3555 - val_loss: 13878.6182 - learning_rate: 0.0010
Epoch 5/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - loss: 20009.2422 - val_loss: 14070.0205 - learning_rate: 0.0010
Epoch 6/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 19485.7188 - val_loss: 15289.6377 - learning_rate: 0.0010
Epoch 7/100
844/847 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 19311.4863
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
847/847 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 19310.0234 - val_loss: 14806.9014 - learning_rate: 0.0010
Epoch 8/100
847/847 ━━━━━━━━

In [3]:
# Function to take user input and make a prediction
def predict_ac_power():
    print("\nEnter feature values for prediction:")

    # Accept user input for all features
    feature_values = []
    for col in features.columns:
        val = float(input(f"Enter {col}: "))
        feature_values.append(val)

    # Convert input to numpy array
    feature_array = np.array(feature_values).reshape(1, -1)

    # Scale input using the previously fitted scaler
    feature_array_scaled = scaler.transform(feature_array)

    # Reshape for LSTM input
    feature_array_reshaped = np.reshape(feature_array_scaled, (1, 1, feature_array_scaled.shape[1]))

    # Make prediction
    predicted_power = model.predict(feature_array_reshaped)

    # Print the prediction
    print(f"\nPredicted AC Power: {predicted_power[0][0]:.2f}")

# Call function for user input and prediction
predict_ac_power()


Enter feature values for prediction:
Enter AMBIENT_TEMPERATURE: 29.91
Enter MODULE_TEMPERATURE: 45.68
Enter IRRADIATION: 0.71
Enter DAILY_YIELD: 2176.6
Enter MONTH: 5
Enter DAY: 29
Enter HOUR: 10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

Predicted AC Power: 908.64


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [4]:
import pickle

In [5]:
import pickle

# Save the trained model
model.save("solar_energy_model.h5")  # Save in HDF5 format

# Save the scaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Model and scaler saved successfully!")


Model and scaler saved successfully!
